<a href="https://colab.research.google.com/github/contactaman5/Bert-Classifier/blob/main/BERT_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!export WANDB_DISABLED=true

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification

from sklearn.metrics import accuracy_score, classification_report

import os
os.environ["WANDB_DISABLED"] = "true"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"  # Suppresses TensorFlow logs
os.environ["WANDB_API_KEY"] = "dummy"  # Prevents API prompts

/usr/local/lib/python3.11/dist-packages/torch_xla/__init__.py:253: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(


## Load Data

In [3]:
data_dir = '/content/drive/MyDrive/Data Science/NLP/data/cleaned data.csv'
data = pd.read_csv(data_dir)

In [4]:
data.head()

,Text,Classification
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [5]:
X = data['Text'].values.tolist()
Y = data['Classification'].values.tolist()

In [6]:
le = LabelEncoder()
y = le.fit_transform(Y)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

## Tokenization Using BERT Tokenizer

In [8]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

X_train= [tokenizer(x, padding='max_length', truncation=True) for x in X_train]
X_test= [tokenizer(x, padding = 'max_length', truncation = True) for x in X_test]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [9]:
num_labels = len(le.classes_)
print("No of classes: ", num_labels)

No of classes:  6


In [10]:
# intialize BERT Model

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased",
                                           num_labels = num_labels)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Define Arguments

In [11]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=200,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [12]:
!pip install --quiet datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.9/274.9 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.0/129.0 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.1/231.1 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.1/344.1 kB 19.7 MB/s eta 0:00:00


In [13]:
from datasets import Dataset

train_dataset = Dataset.from_dict({"input_ids": [x["input_ids"] for x in X_train],
                                  "attention_mask": [x["attention_mask"] for x in X_train],
                                  "labels": y_train})
eval_dataset = Dataset.from_dict({"input_ids": [x["input_ids"] for x in X_test],
                                 "attention_mask": [x["attention_mask"] for x in X_test],
                                 "labels": y_test})

In [14]:
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

import numpy as np
# from datasets import load_metric


# metric = load_metric("accuracy")

# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     predictions = np.argmax(logits, axis=-1)
#     return metric.compute(predictions=predictions, references=labels)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    # compute_metrics=compute_metrics,
)



<ipython-input-14-541061e4d2df>:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss


In [ ]:
trainer.evaluate()

In [ ]:
texts = [
    "The movie was fantastic! I really enjoyed it.",
    "I hated the food, it was terrible.",
    "The product is okay, not the best but not the worst.",
    "Absolutely loved the experience! Highly recommend.",
    "Worst service ever. Never coming back."
]

# Tokenize the entire list
inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)

# Get predictions
output = model(**inputs)
predicted_classes = output.logits.argmax(dim=1).tolist()

# Convert predictions to labels
sentiments = ["Positive" if pred == 1 else "Negative" for pred in predicted_classes]

# Print results
for text, sentiment in zip(texts, sentiments):
    print(f"Text: {text}\nSentiment: {sentiment}\n")